# Nuage Compta

In [1]:
import modules.images.utils as im

## Extraction des images

La première partie consiste à extraire les lettres de l'image de test

In [3]:
import cv2
import numpy as np

image = cv2.imread("test.jpeg")

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
cv2.imwrite("../data/staging/gray.jpeg", gray)

# Permet de convertir l'image en blanc sur fond noir. Etape nécessaire pour la méthode findContours de OpenCV et par rapport à notre cible
_, threshold = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU + cv2.THRESH_BINARY_INV) 
cv2.imwrite("../data/staging/threshold.jpeg", threshold)

# Copie de l'image sur laquelle on va travailler.
target_image = threshold.copy()

# Detection des contours. On demande de prendre que les points externes avec RETR_EXTERNAL et d'optimiser le nombre de points avec CHAIN_APPROX_SIMPLE
contours = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]

roi_index = 0

for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)

    if w < 50 and h < 50:
        continue

    # Extraction de la lettre dans l'image
    roi = target_image[y:y+h, x:x+w]
    cv2.imwrite('../data/staging/extract/region_{}.jpeg'.format(roi_index), roi)

    # Resize de la lettre pour être en 20x20 en gardant le ratio
    r_img = im.resize_image(roi)
    cv2.imwrite('../data/staging/resize/region_resize_{}.jpeg'.format(roi_index), r_img)

    # Ajout du padding de 4
    p_img= cv2.copyMakeBorder(r_img.copy(),4,4,4,4,cv2.BORDER_CONSTANT)
    cv2.imwrite('../data/out/region_padded_{}.jpeg'.format(roi_index), p_img)

    roi_index += 1